#**Matching & Unmatching**

##Part 1

In this section, you'll need to supply the Excel file containing the responses from the Microsoft Form. The system will parse the email addresses to extract student IDs. Following this, it will generate a new Excel file named "Processed_Responses.xlsx". This file will consist of two sheets:

1. **'student' sheet:** Contains the extracted IDs of students.
2. **'others' sheet:** Contains details of all individuals who filled the form but are not identified as students.

P.S ALL THE FILES WILL BE DOWNLOADED AUTOMATICALLY ONCE YOU PROVIDE THE NECESSARY INPUT AS PER INSTRUCTIONS


In [ ]:
import pandas as pd
from openpyxl import Workbook
from google.colab import files

# Upload the Excel file
uploaded = files.upload()

# Define the path for the uploaded file
form_responses_path = list(uploaded.keys())[0]

# Load the Excel file
form_responses = pd.read_excel(form_responses_path)

# Function to extract ID from email
def extract_id(email):
    if email.endswith('@st.habib.edu.pk'):
        return int(email.split('@')[0][2:].lstrip('0'))
    return None

# Extract IDs
form_responses['ID'] = form_responses['Email'].apply(extract_id)

# Separate valid IDs and others
students_df = form_responses.dropna(subset=['ID'])[['ID', 'Name']]
others_df = form_responses[form_responses['ID'].isna()][['Name', 'Email']]

# Convert IDs to integer format
students_df['ID'] = students_df['ID'].astype(int)

# Create a new Excel file with two sheets: "students" and "others"
wb = Workbook()
students_sheet = wb.active
students_sheet.title = "students"

# Add headers and data to the "students" sheet
students_sheet.append(['ID', 'Name'])
for index, row in students_df.iterrows():
    students_sheet.append([row['ID'], row['Name']])

# Create "others" sheet
others_sheet = wb.create_sheet(title='others')

# Add headers and data to the "others" sheet
others_sheet.append(['Name', 'Email'])
for index, row in others_df.iterrows():
    others_sheet.append([row['Name'], row['Email']])

# Save the workbook
output_path = 'Processed_Responses.xlsx'
wb.save(output_path)

# Download the updated file
files.download(output_path)

Saving Petition for Palestine(1-420).xlsx to Petition for Palestine(1-420).xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Part 2

In this section, you'll need to provide a file named "cleaned_Batchnames.xlsx" which contains batch years (separate sheets) along with student IDs and names, categorized batch-wise. The output file you will get will be used in Part 3.


In [ ]:
import pandas as pd
from google.colab import files

#Data Cleaning for batch names

# Upload the Excel file
uploaded = files.upload()

# Define the path for the uploaded file
batch_names_path = list(uploaded.keys())[0]

# Load the Excel file with all sheets
batch_names_df = pd.read_excel(batch_names_path, sheet_name=None)

# Function to clean and convert ID
def clean_convert_id(id_value):
    if pd.isna(id_value):
        return id_value
    try:
        # Remove any leading/trailing whitespace
        id_str = str(id_value).strip()
        # Convert to integer if possible
        if id_str.isdigit():
            return int(id_str)
        return id_value
    except ValueError:
        return id_value

# Process each sheet
cleaned_sheets = {}
for sheet_name, df in batch_names_df.items():
    if 'ID' in df.columns:
        # Apply the function to clean and convert IDs
        df['ID'] = df['ID'].apply(clean_convert_id)
    cleaned_sheets[sheet_name] = df

# Save the cleaned data to a new Excel file
output_path = 'Cleaned_Batch_Names.xlsx'
with pd.ExcelWriter(output_path) as writer:
    for sheet_name, df in cleaned_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Download the updated file
files.download(output_path)
print("Cleaned data saved to", output_path)


Saving Batch_Names.xlsx to Batch_Names.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Cleaned data saved to Cleaned_Batch_Names.xlsx


##Part 3

To complete this task, you'll need to provide the **"processed_responses.xlsx"** file from part 1, and **"cleaned_Batchnames.xlsx"** file from part 2, also obtained from previous steps. The output will consist of a file containing information about individuals who aren't students and their respective statuses, along with matched and unmatched statuses for students grouped by batch.

It will be named as **"Modified_Batch_Names.xlsx"**


**P.S: left_only: The row's key was found only in the left (first) DataFrame. So, left_only means that the IDs in those rows were found only in the batch_names file but not in the processed_responses file.**

In [ ]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill
from google.colab import files
from io import BytesIO

# Function to process the files
def process_files(processed_responses_file, batch_names_file):
    # Read the batch names file
    batch_names = pd.read_excel(batch_names_file, sheet_name=None)
    print(f"Batch names sheets: {list(batch_names.keys())}")

    # Read the processed responses file
    processed_responses = pd.read_excel(processed_responses_file, sheet_name=None)
    print(f"Processed responses sheets: {list(processed_responses.keys())}")

    # Extract student data and others data
    if 'students' in processed_responses:
        student_data = processed_responses['students']
        print(f"Students sheet columns: {student_data.columns.tolist()}")
    else:
        raise KeyError("The 'students' sheet is not found in the processed responses file.")

    if 'others' in processed_responses:
        others_data = processed_responses['others']
        print(f"Others sheet columns: {others_data.columns.tolist()}")
    else:
        raise KeyError("The 'others' sheet is not found in the processed responses file.")

    # Create a workbook for the output
    wb = Workbook()
    ws_others = wb.create_sheet(title="Others")

    # Iterate through each sheet in batch_names
    for sheet_name, batch_data in batch_names.items():
        if 'ID' in batch_data.columns and 'Name' in batch_data.columns:
            # Merge with student data
            merged_data = batch_data.merge(student_data[['ID']], on='ID', how='left', indicator=True)

            # Add 'Status' column
            merged_data['Status'] = 'Unmatched'
            merged_data.loc[merged_data['_merge'] == 'both', 'Status'] = 'Matched'

            # Create a new worksheet in the output workbook
            ws = wb.create_sheet(title=sheet_name)

            for r_idx, row in merged_data.iterrows():
                ws.append(row.tolist())
                if row['Status'] == 'Matched':
                    ws.cell(row=r_idx + 2, column=len(merged_data.columns)).fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
                else:
                    ws.cell(row=r_idx + 2, column=len(merged_data.columns)).fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')

    # Merge the 'Others' sheet into the new Excel file
    for r in dataframe_to_rows(others_data, index=False, header=True):
        ws_others.append(r)

    # Remove the default sheet created by Workbook
    wb.remove(wb['Sheet'])

    # Save the workbook to a bytes buffer
    output = BytesIO()
    wb.save(output)
    output.seek(0)

    # Save the buffer to a file and download it
    with open("Modified_Batch_Names.xlsx", "wb") as f:
        f.write(output.getvalue())

    files.download("Modified_Batch_Names.xlsx")

# Prompt user to upload the first Excel file (processed_responses.xlsx)
print("Please upload the first Excel file (processed_responses.xlsx):")
uploaded1 = files.upload()

# Get the uploaded file name
processed_responses_file = next(iter(uploaded1))

# Prompt user to upload the second Excel file (batch_names.xlsx)
print("\nPlease upload the second Excel file (batch_names.xlsx):")
uploaded2 = files.upload()

# Get the uploaded file name
batch_names_file = next(iter(uploaded2))

# Process the files
process_files(processed_responses_file, batch_names_file)


Please upload the first Excel file (processed_responses.xlsx):


Saving Processed_Responses (1).xlsx to Processed_Responses (1).xlsx

Please upload the second Excel file (batch_names.xlsx):


Saving Cleaned_Batch_Names (1).xlsx to Cleaned_Batch_Names (1).xlsx
Batch names sheets: ['2025', '2026', '2027']
Processed responses sheets: ['students', 'others']
Students sheet columns: ['ID', 'Name']
Others sheet columns: ['Name', 'Email']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>